# Parse ariel3_thunderstorm-noise

This is a quick Python example of parsing a tiny part of the relatively complicated Ariel3 satellite thunderstorm-noise data.

* Inspired by breqm76
* File format documentation at
https://docs.google.com/document/d/1GioN-8mch3GeruMrqG6qVLklQRDOh1TiARfYwI7BC4k/edit#heading=h.uydog7egy1sz
* Parsing the files at https://spdf.sci.gsfc.nasa.gov/pub/data/ariel/ariel3/langmuir_probe/plasma-frequency_electron-temperature_thunderstorm-noise/DATA2_DR002129_DR002129_20080804_091423/


In [1]:
def chunkstring(string, length):
    "Return the string, split up into fixed-width substrings of given length"
    return (string[0+i:length+i] for i in range(0, len(string), length))

def groupOf10(data, offset):
    "print the data as 10 sets of three 5-character numbers"    
    segment = data[offset:offset+10*3*5]
    print('\n'.join(chunkstring(' '.join(chunkstring(segment[:30*5],5)), 18)))

In [2]:
# File is encoded as EBCDIC (codec cp500)
with open('dr002129_f00001.phys.1', 'rt', encoding='cp500') as ariel3_thunderstorm_noise:
    d2129 = ariel3_thunderstorm_noise.read()

In [3]:
groupOf10(d2129, 0x11F02)

 -6.1  -9.0 -14.3 
 -6.2  -9.1 -14.4 
 -6.5  -9.1 -14.4 
 -6.8  -9.2 -14.5 
 -7.0  -9.3 -14.6 
 -7.2  -9.4 -14.7 
 -7.4  -9.5 -14.8 
 -7.6  -9.7 -14.9 
 -7.8  -9.8 -15.0 
 -8.0  -9.9 -15.1


In [4]:
groupOf10(d2129, 0x11FA6)

 -8.1 -10.0 -15.2 
 -8.2 -10.1 -15.3 
 -8.3 -10.2 -15.4 
 -8.4 -10.3 -15.5 
 -8.4 -10.4 -15.6 
 -8.5 -10.5 -15.7 
 -8.5 -10.6 -15.8 
 -8.6 -10.7 -15.9 
 -8.8 -10.8 -16.0 
 -9.0 -10.9 -16.1


# Now we just start looking and playing around 

Fields in each record of section 1, "5 MHz or 15MHz data records"

F4.0, A2, 3F3.0, F5.2, F7.3, F8.3, F5.1, F4.0, F3.0, F5.2, F4.1, F5.2, 2F6.1, 2(F6.1, F4.1), F61, F5.2, 3F5.0, 3F6.2

In [5]:
# estimate length of records in section 1
4 + 2 + 9 + 5 + 7 + 8 + 5 + 4 + 3 + 5 + 4 + 5 + 12 + 2*(6+4) + 61 + 5 + 3*5 + 3*6

192

In [6]:
headerLength = 174 # empirically noticed repeat of pairs of records with this chunking
header = d2129[:headerLength]

offset = headerLength
sOneLength = 164
M = 424

section1 = d2129[offset:offset+(M * sOneLength)]


In [7]:
header

'\x00s                 131    4                 MAD                     0                                                                                     0000000000\x00s  1.  15'

Print the first half or so of each record in what seem to be pretty regular repeats in section 1. There seems to be a smaller interval after the odd records than after the even ones

In [8]:
["%4d %s" % (i, l[:90]) for i, l in enumerate(list(chunkstring(section1, 164)))]

['   0 . 3. 2.23.20 -1.357  69.581506.5  7.40.42.8010.9 3.672100.0  20.9 -15.7 0.0 -15.7 0.0 283.',
 '   1  2.26.70 -1.577  69.605506.6  7.40.51.8010.9 3.672100.0  20.9 -10.4 0.0 -10.4 0.0 283.0 0.',
 '   2 . 3. 2.51.10 -3.116  69.768507.5  7.41.55.5010.8 3.762100.0  20.9 -15.8 0.0 -15.8 0.0 279.',
 '   3  2.54.60 -3.336  69.792507.6  7.42. 4.6010.8 3.762100.0  20.9 -10.5 0.0 -10.5 0.0 278.0 0.',
 '   4 . 3. 3.19.10 -4.874  69.955508.6  7.43. 8.3010.6 3.792100.0  21.1 -15.9 0.0 -15.9 0.0 275.',
 '   5  3.22.60 -5.094  69.979508.7  7.43.17.4010.6 3.792100.0  21.1 -10.6 0.0 -10.6 0.0 274.0 0.',
 '   6 . 3. 3.47.00 -6.631  70.144509.7  7.44.21.5010.4 3.812100.0  21.1 -16.0 0.0 -16.0 0.0 271.',
 '   7  3.50.50 -6.851  70.167509.9  7.44.30.6010.3 3.812100.0  21.1 -10.7 0.0 -10.7 0.0 271.0 0.',
 '   8 . 3. 4.14.90 -8.388  70.334510.9  7.45.35.1010.1 3.762100.0  20.9 -16.1 0.0 -16.1 0.0 269.',
 '   9  4.18.40 -8.607  70.358511.1  7.45.44.3010.0 3.762100.0  20.9 -10.8 0.0 -10.8 0.0 268.0 0.',


In [9]:
gap = '. 0. 0. 0.00  0.000   0.000  0.0  0. 0. 0.00 0.0 0.00   0.0   0.0   0.0 0.0   0.0 0.0   0.0 0.00   0.   0.   0.  0.00  0.00  0.00                       00\x00s  25                                                                                                                                                            00\x00s    2.2  -71.4                                                                                                                                                  00\x00s     2   212                                                                                                                                                    00\x00s '

In [10]:
len(gap)

649

In [11]:

offset = headerLength + M * sOneLength + 677
sTwoLength = 10 * 3 * 5
section2 = d2129[offset:offset+(M * sTwoLength)]


In [12]:
groupOf10(section2, sTwoLength)

 -9.3 -11.5 -16.8 
 -9.5 -11.6 -16.8 
 -9.5 -11.7 -16.9 
 -9.5 -11.7 -17.0 
 -9.6 -11.8 -17.0 
 -9.7 -11.8 -17.1 
 -9.9 -11.9 -17.1 
 -9.9 -11.9 -17.2 
 -9.9 -12.0 -17.2 
            00 s-


In [13]:
groupOf10(section2[sTwoLength:], sTwoLength)

10.0- 12.0- 17.2- 
10.1- 12.1- 17.3- 
10.1- 12.1- 17.3- 
10.2- 12.1- 17.3- 
10.3- 12.1- 17.4- 
10.2- 12.1- 17.4- 
10.2- 12.2- 17.4- 
10.2- 12.2- 17.4- 
10.1- 12.1- 17.4- 
10.1- 12.2- 17.4 
